In [25]:
from langflow.load import run_flow_from_json
import json
import os

In [26]:
def load_flow(path):
    with open(path, "r") as f:
        return json.load(f)

In [27]:
def print_agent_stdouts(result, show_stderr=True):
    """
    result: the object returned by run_flow_from_json(...)
    show_stderr: set True to also print stderr blocks
    """
    # 1) Extract the JSON string produced by ChemPipeline component
    text = result[0].outputs[0].results["message"].data["text"]
    payload = json.loads(text)

    # 2) Normalize runs list (supports both python_runs and legacy python_run)
    runs = payload.get("python_runs") or []
    if not runs and payload.get("python_run"):  # legacy single
        runs = [payload["python_run"]]

    if not runs:
        print("No python runs found in the agent output.")
        return payload  # return in case you want to inspect other fields

    # 3) Pretty print stdout (and optionally stderr) for each run
    sep = "_" * 96
    for i, r in enumerate(runs, 1):
        cmd = r.get("cmd", "<no cmd>")
        rc = r.get("rc", None)
        stdout = r.get("stdout", "") or ""
        stderr = r.get("stderr", "") or ""

        if 'reduce.py' in cmd:
            job = 'REDUCTION CANDIDATE SEARCH'
        elif 'conformer.py' in cmd:
            job = 'CONFORMER SEARCH'
        elif 'generate_nw.py' in cmd:
            job = 'SCRIPT GENERATION'
        print(sep)
        print(f"[{job}]\n rc={rc}  cmd: {cmd}")
        print("\n--- Output ---")
        print(stdout if stdout.strip() else "")
        if show_stderr:
            print("\n")
            print(stderr if stderr.strip() else "")
    print(sep)

In [28]:
# Local Code Repo Location
os.environ["REDOXGIT"] = "/home/cameron/Documents/Github/RedoxFlow"

In [29]:
flow_json_path = f"{os.environ['REDOXGIT']}/scripts/redoxflow.json"

In [35]:
prompt = {
  "count": 100,
  "max_atoms": 16,
  "min_unique_elements": 1,
  "elements": "CNOF",

  "temperature": 0.425,
  "top_p": 0.9,
  "repetition_penalty": 1.10,
  "num_return_sequences": 96,

  "device": "auto",

  "python_jobs": [
    {
      "script": f"{os.environ['REDOXGIT']}/scripts/reduce.py",
      "args": [
        "--molecules",f"{os.environ['REDOXGIT']}/memory/reactants_memory.csv",
        "--reactions", f"{os.environ['REDOXGIT']}/scripts/reactions.json",
        "--out", f"{os.environ['REDOXGIT']}/memory/reduction_candidates.csv",
        "--depth", "2"
      ]
    },
    {
      "script": f"{os.environ['REDOXGIT']}/scripts/conformer.py",
      "args": [
        "--in", f"{os.environ['REDOXGIT']}/memory/reduction_candidates.csv",
        "--out", f"{os.environ['REDOXGIT']}/memory/products_memory.csv",
        "--max-confs", "200",
        "--min-steps", "1000",
        "--seed", "1337",
        "--batch-size", "1000"
      ]
    },
    {
      "script": f"{os.environ['REDOXGIT']}/scripts/generate_nw.py",
      "args": [
        "--reactants_in", f"{os.environ['REDOXGIT']}/memory/reactants",
        "--products_in", f"{os.environ['REDOXGIT']}/memory/products",
        "--scripts_out", f"{os.environ['REDOXGIT']}/memory/scripts"
        ]
    }
  ],

  "memory_path": f"{os.environ['REDOXGIT']}/memory/reactants_memory.csv"
}


In [36]:
# 1) Load flow dict
flow = load_flow(flow_json_path)

In [37]:
from rdkit import RDLogger
from rdkit import rdBase

RDLogger.DisableLog("rdApp.*")   # suppress error/warn/info
rdBase.DisableLog("rdApp.*")     # some RDKit builds use this hook too

In [40]:
# 3) Run (disable cache to avoid any stale flow)
result = run_flow_from_json(flow, input_value=json.dumps(prompt), cache=False, disable_logs=True)

In [41]:
print_agent_stdouts(result)

________________________________________________________________________________________________
[REDUCTION CANDIDATE SEARCH]
 rc=0  cmd: /home/cameron/Documents/envs/chemflow/bin/python /home/cameron/Documents/Github/RedoxFlow/scripts/reduce.py --molecules /home/cameron/Documents/Github/RedoxFlow/memory/reactants_memory.csv --reactions /home/cameron/Documents/Github/RedoxFlow/scripts/reactions.json --out /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv --depth 2

--- Output ---
Wrote 274 rows to /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv




________________________________________________________________________________________________
[CONFORMER SEARCH]
 rc=0  cmd: /home/cameron/Documents/envs/chemflow/bin/python /home/cameron/Documents/Github/RedoxFlow/scripts/conformer.py --in /home/cameron/Documents/Github/RedoxFlow/memory/reduction_candidates.csv --out /home/cameron/Documents/Github/RedoxFlow/memory/products_memory.csv --max-